In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats
from io import StringIO

In [5]:
# datetime cleaning
_ = """
time,other_col
01-01-2000 12:00:00,5
02-01-2000 13:00:00,6
"""
df = pd.read_csv(StringIO(_))

df["time"] = pd.to_datetime(df["time"], format="%d-%m-%Y %H:%M:%S")
df

,time,other_col
0,2000-01-01 12:00:00,5
1,2000-01-02 13:00:00,6


In [ ]:
# time zone localize: assuming naive datetime is in local time
df["time_sydney"] = df["time"].dt.tz_localize("Australia/Sydney")
df["time_singapore"] = df["time"].dt.tz_localize("Asia/Singapore")
df

,time,other_col,time_sydney,time_singapore
0,2000-01-01 12:00:00,5,2000-01-01 12:00:00+11:00,2000-01-01 12:00:00+08:00
1,2000-01-02 13:00:00,6,2000-01-02 13:00:00+11:00,2000-01-02 13:00:00+08:00


In [10]:
# time zone convert to UTC then remove time zone info
df["time_sydney_utc_naive"] = df["time_sydney"].dt.tz_convert("UTC").dt.tz_localize(None)
df


,time,other_col,time_sydney,time_singapore,time_sydney_utc_naive
0,2000-01-01 12:00:00,5,2000-01-01 12:00:00+11:00,2000-01-01 12:00:00+08:00,2000-01-01 01:00:00
1,2000-01-02 13:00:00,6,2000-01-02 13:00:00+11:00,2000-01-02 13:00:00+08:00,2000-01-02 02:00:00
